In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
from scipy import signal as sp_signal
from scipy import interpolate
from scipy.ndimage import gaussian_filter1d
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Config
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Paths
DATA_PATH = Path('/kaggle/input/physionet-ecg-image-digitization')
TRAIN_PATH = DATA_PATH / 'train'
TEST_PATH = DATA_PATH / 'test'

print("🚀 PhysioNet ECG - BASELINE V3: Adaptive Detection")
print("=" * 70)
print(f"📁 Data loaded: {DATA_PATH.exists()}")

# Load metadata
train_meta = pd.read_csv(DATA_PATH / 'train.csv')
test_meta = pd.read_csv(DATA_PATH / 'test.csv')

print(f"📊 Train samples: {len(train_meta)}")
print(f"📊 Test samples: {len(test_meta['id'].unique())}")

In [ ]:
CALIBRATION = {
    'px_per_mm': 39,
    'px_per_second': 975,
    'px_per_mv': 390,
}

ECG_LEADS_ORDER = ['I', 'aVR', 'V1', 'V4', 'II', 'aVL', 'V2', 'V5', 
                   'III', 'aVF', 'V3', 'V6']

print("\n" + "="*70)
print("⚙️  CONFIGURATION")
print("="*70)
print(f"📐 Calibration: {CALIBRATION['px_per_mv']} px/mV")
print(f"📋 Lead order: {ECG_LEADS_ORDER}")

In [ ]:
print("\n" + "="*70)
print("🔧 STEP 1: ROBUST DESKEW")
print("="*70)

def detect_skew_angle(img, visualize=False):
    """
    Detect rotation angle using Hough Line Transform
    
    Args:
        img: Grayscale image
        visualize: Show detection process
    
    Returns:
        angle: Rotation angle in degrees
        debug_info: Dict with visualization data
    """
    # Edge detection
    edges = cv2.Canny(img, 50, 150, apertureSize=3)
    
    # Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=200)
    
    if lines is None:
        return 0.0, {'edges': edges, 'lines': []}
    
    # Extract angles
    angles = []
    line_coords = []
    
    for rho, theta in lines[:, 0]:
        angle = np.degrees(theta) - 90
        # Keep only near-horizontal lines
        if -10 < angle < 10:
            angles.append(angle)
            # Store line coordinates for visualization
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            line_coords.append((x0, y0, theta))
    
    # Calculate median angle (robust to outliers)
    median_angle = np.median(angles) if len(angles) > 0 else 0.0
    
    debug_info = {
        'edges': edges,
        'lines': line_coords,
        'angles': angles,
        'median_angle': median_angle
    }
    
    return median_angle, debug_info


def deskew_image(img, angle, min_angle_threshold=1.0):
    """
    Rotate image to correct skew ONLY if significant
    
    Args:
        img: Input image
        angle: Rotation angle in degrees
        min_angle_threshold: Minimum angle to bother rotating
    
    Returns:
        Deskewed image (or original if angle too small)
    """
    # Don't rotate if angle is negligible
    if abs(angle) < min_angle_threshold:
        print(f"   ℹ️  Skew angle ({angle:.3f}°) below threshold, keeping original")
        return img
    
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # Calculate new image size to avoid cropping
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    
    # Adjust transformation matrix
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    
    deskewed = cv2.warpAffine(img, M, (new_w, new_h),
                              flags=cv2.INTER_LINEAR,
                              borderMode=cv2.BORDER_CONSTANT,
                              borderValue=255)
    
    print(f"   ✅ Rotated {angle:.3f}° to correct skew")
    return deskewed


def visualize_deskew(img_original, img_deskewed, debug_info):
    """Visualize deskew process"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Original
    axes[0, 0].imshow(img_original, cmap='gray')
    axes[0, 0].set_title('1. Original Image', fontweight='bold', fontsize=12)
    axes[0, 0].axis('off')
    
    # Edges
    axes[0, 1].imshow(debug_info['edges'], cmap='gray')
    axes[0, 1].set_title('2. Edge Detection (Canny)', fontweight='bold', fontsize=12)
    axes[0, 1].axis('off')
    
    # Detected lines
    img_lines = cv2.cvtColor(img_original, cv2.COLOR_GRAY2BGR)
    for x0, y0, theta in debug_info['lines'][:20]:  # Show first 20 lines
        length = 1000
        x1 = int(x0 + length * (-np.sin(theta)))
        y1 = int(y0 + length * (np.cos(theta)))
        x2 = int(x0 - length * (-np.sin(theta)))
        y2 = int(y0 - length * (np.cos(theta)))
        cv2.line(img_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    axes[1, 0].imshow(img_lines)
    axes[1, 0].set_title(f'3. Detected Lines (angle: {debug_info["median_angle"]:.2f}°)', 
                        fontweight='bold', fontsize=12)
    axes[1, 0].axis('off')
    
    # Deskewed
    axes[1, 1].imshow(img_deskewed, cmap='gray')
    axes[1, 1].set_title('4. Deskewed Image', fontweight='bold', fontsize=12)
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Detected skew: {debug_info['median_angle']:.3f}°")
    print(f"   Original size: {img_original.shape}")
    print(f"   Deskewed size: {img_deskewed.shape}")

print("✅ Deskew functions defined")

In [ ]:
print("\n" + "="*70)
print("🔧 STEP 2: ADAPTIVE ROW DETECTION")
print("="*70)

def detect_content_start(img, threshold=0.03):
    """
    Detect where ECG content starts (skip header region)
    
    Args:
        img: Grayscale image
        threshold: Min ratio of dark pixels to consider as content
    
    Returns:
        content_start_y: Y-coordinate where content begins
        debug_info: Visualization data
    """
    h, w = img.shape
    
    # Analyze each horizontal strip (10px high)
    strip_height = 10
    n_strips = h // strip_height
    
    content_ratios = []
    strip_positions = []
    
    for i in range(n_strips):
        y_start = i * strip_height
        y_end = min((i + 1) * strip_height, h)
        strip = img[y_start:y_end, :]
        
        # Calculate ratio of dark pixels (ECG signal)
        dark_pixels = np.sum(strip < 200)
        total_pixels = strip.size
        dark_ratio = dark_pixels / total_pixels
        
        content_ratios.append(dark_ratio)
        strip_positions.append(y_start)
    
    content_ratios = np.array(content_ratios)
    strip_positions = np.array(strip_positions)
    
    # Find first strip with significant content
    content_mask = content_ratios > threshold
    
    if np.any(content_mask):
        first_content_idx = np.argmax(content_mask)
        content_start_y = strip_positions[first_content_idx]
    else:
        content_start_y = 0
    
    debug_info = {
        'content_ratios': content_ratios,
        'strip_positions': strip_positions,
        'threshold': threshold,
        'content_start': content_start_y
    }
    
    return content_start_y, debug_info


def detect_and_filter_text_regions(img):
    """
    Detect and mask text regions (labels like 'aVR', 'V1', etc.)
    Text has different characteristics than ECG signals
    
    Args:
        img: Grayscale image
    
    Returns:
        mask: Binary mask (0 = text, 255 = keep)
    """
    # Text detection using morphological operations
    # Text has small, dense connected components
    
    # Threshold to get dark pixels
    _, binary = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Find connected components
    n_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)
    
    # Create mask (start with all white = keep everything)
    mask = np.ones_like(img) * 255
    
    # Filter out small, compact regions (likely text)
    for i in range(1, n_labels):  # Skip background (0)
        x, y, w, h, area = stats[i]
        
        # Text characteristics:
        # - Small area (< 500 pixels)
        # - Compact shape (w and h both small)
        # - Usually in margins (x < 200 or x > img.shape[1] - 200)
        
        aspect_ratio = max(w, h) / (min(w, h) + 1)
        
        is_likely_text = (
            area < 500 and                    # Small area
            w < 80 and h < 50 and            # Compact size
            aspect_ratio < 5                  # Not too elongated (ECG is very elongated)
        )
        
        if is_likely_text:
            # Mark this region as text (set to 0 in mask)
            mask[labels == i] = 0
    
    return mask


def detect_rows_smart_v2(img, n_expected_rows=4):
    """
    Improved smart row detection with text filtering
    
    Args:
        img: Deskewed image
        n_expected_rows: Expected number of content rows (typically 3-4)
    
    Returns:
        rows: List of (y_start, y_end, row_type)
        debug_info: Visualization data
    """
    h, w = img.shape
    
    # Step 1: Filter out text regions
    text_mask = detect_and_filter_text_regions(img)
    img_filtered = cv2.bitwise_and(img, img, mask=text_mask)
    
    # Step 2: Find where content starts (skip header)
    content_start, debug_content = detect_content_start(img_filtered, threshold=0.03)
    
    # Step 3: Analyze content region WITHOUT text
    content_img = img_filtered[content_start:, :]
    content_h = content_img.shape[0]
    
    # Project to y-axis
    y_projection = np.mean(content_img, axis=1)
    y_projection_smooth = gaussian_filter1d(y_projection, sigma=30)  # Heavy smoothing
    
    # Invert
    y_projection_inverted = 255 - y_projection_smooth
    y_proj_norm = (y_projection_inverted - y_projection_inverted.min()) / \
                  (y_projection_inverted.max() - y_projection_inverted.min())
    
    # Find peaks with very strict parameters
    from scipy.signal import find_peaks
    peaks, properties = find_peaks(y_proj_norm, 
                                   distance=200,      # Very far apart
                                   prominence=0.2,    # Very prominent
                                   width=100)         # Very wide
    
    # If we got too many or too few peaks, use proportional division
    if len(peaks) < 2 or len(peaks) > 5:
        print(f"   ⚠️  Peak detection gave {len(peaks)} peaks, using proportional division instead")
        
        # Use proportional division based on expected structure
        # Typically: 3 equal rows + 1 larger rhythm strip
        # Or: 3 equal rows (if rhythm integrated)
        
        if n_expected_rows == 4:
            # 3 rows of ~20% each + 1 rhythm of ~40%
            proportions = [0.20, 0.20, 0.20, 0.40]
        else:
            # 3 equal rows
            proportions = [1/3, 1/3, 1/3]
        
        boundaries = [0]
        cumulative = 0
        for prop in proportions:
            cumulative += prop
            boundaries.append(int(content_h * cumulative))
        boundaries[-1] = content_h  # Ensure last goes to end
        
        rows = []
        for i in range(len(boundaries) - 1):
            y_start = content_start + boundaries[i]
            y_end = content_start + boundaries[i + 1]
            row_height = y_end - y_start
            
            # Last row is typically rhythm if it's significantly larger
            if i == len(boundaries) - 2 and row_height > content_h * 0.35:
                row_type = 'rhythm'
            else:
                row_type = 'standard'
            
            rows.append((y_start, y_end, row_type))
    
    else:
        # Use detected peaks
        boundaries = [0]
        for i in range(len(peaks) - 1):
            valley_region = y_proj_norm[peaks[i]:peaks[i+1]]
            valley_idx = np.argmin(valley_region)
            valley_pos = peaks[i] + valley_idx
            boundaries.append(valley_pos)
        boundaries.append(content_h)
        
        rows = []
        for i in range(len(boundaries) - 1):
            y_start = content_start + boundaries[i]
            y_end = content_start + boundaries[i + 1]
            row_height = y_end - y_start
            
            # Determine row type
            if row_height > content_h * 0.35:
                row_type = 'rhythm'
            else:
                row_type = 'standard'
            
            rows.append((y_start, y_end, row_type))
    
    debug_info = {
        'content_start': content_start,
        'y_projection': y_proj_norm,
        'peaks': peaks,
        'content_ratios': debug_content['content_ratios'],
        'strip_positions': debug_content['strip_positions'],
        'text_mask': text_mask
    }
    
    return rows, debug_info


def visualize_row_detection(img, rows, debug_info):
    """Visualize smart row detection process"""
    fig = plt.figure(figsize=(20, 10))
    gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)
    
    # 1. Content detection
    ax1 = fig.add_subplot(gs[0, 0])
    strip_positions = debug_info['strip_positions']
    content_ratios = debug_info['content_ratios']
    content_start = debug_info['content_start']
    
    ax1.plot(content_ratios, strip_positions, linewidth=2, color='blue')
    ax1.axhline(content_start, color='red', linestyle='--', linewidth=2, 
               label=f'Content starts at y={content_start}')
    ax1.invert_yaxis()
    ax1.set_xlabel('Content Ratio', fontweight='bold')
    ax1.set_ylabel('Y Position (pixels)', fontweight='bold')
    ax1.set_title('1. Header Detection', fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 2. Y-projection with peaks
    ax2 = fig.add_subplot(gs[0, 1])
    y_proj = debug_info['y_projection']
    peaks = debug_info['peaks']
    
    y_positions = np.arange(len(y_proj)) + content_start
    ax2.plot(y_proj, y_positions, linewidth=2, color='darkgreen')
    
    # Mark peaks
    if len(peaks) > 0:
        peak_positions = peaks + content_start
        peak_values = y_proj[peaks]
        ax2.plot(peak_values, peak_positions, 'ro', markersize=10, 
                label=f'{len(peaks)} peaks detected')
    
    ax2.invert_yaxis()
    ax2.set_xlabel('Content Density (normalized)', fontweight='bold')
    ax2.set_ylabel('Y Position (pixels)', fontweight='bold')
    ax2.set_title('2. Content Density Analysis', fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Detected rows overlaid
    ax3 = fig.add_subplot(gs[0, 2])
    img_rows = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    colors = {
        'standard': (0, 255, 0),  # Green
        'rhythm': (255, 0, 0)     # Red
    }
    
    for idx, (y_start, y_end, row_type) in enumerate(rows):
        color = colors.get(row_type, (0, 255, 255))
        cv2.rectangle(img_rows, (0, y_start), (img.shape[1], y_end), color, 3)
        
        label = f'Row {idx+1} ({row_type})'
        cv2.putText(img_rows, label, (10, y_start + 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    
    # Mark content start
    cv2.line(img_rows, (0, content_start), (img.shape[1], content_start), 
            (0, 0, 255), 2)
    cv2.putText(img_rows, 'Content Start', (10, content_start - 10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    ax3.imshow(img_rows)
    ax3.set_title(f'3. Detected Rows ({len(rows)})', fontweight='bold')
    ax3.axis('off')
    
    # 4. Individual rows (bottom row, spans all columns)
    ax4 = fig.add_subplot(gs[1, :])
    
    # Create composite showing all rows side by side
    row_heights = [y_end - y_start for y_start, y_end, _ in rows]
    max_height = max(row_heights)
    
    composite_width = 0
    row_images = []
    
    for y_start, y_end, row_type in rows:
        row_img = img[y_start:y_end, :]
        # Resize to same height for display
        scale = max_height / row_img.shape[0]
        new_width = int(row_img.shape[1] * scale)
        row_resized = cv2.resize(row_img, (new_width, max_height))
        row_images.append(row_resized)
        composite_width += new_width
    
    # Concatenate horizontally
    composite = np.hstack(row_images)
    
    ax4.imshow(composite, cmap='gray')
    ax4.set_title('4. All Rows (normalized height for comparison)', fontweight='bold')
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Smart Row Detection Summary:")
    print(f"   Header region: y=0-{content_start}")
    print(f"   Content region: y={content_start}-{img.shape[0]}")
    print(f"   Rows detected: {len(rows)}")
    for idx, (y_start, y_end, row_type) in enumerate(rows):
        height = y_end - y_start
        print(f"   Row {idx+1} ({row_type:>8}): y={y_start:>4}-{y_end:>4} (height: {height:>3}px)")
    
    # Calculate expected number of leads
    standard_rows = sum(1 for _, _, t in rows if t == 'standard')
    rhythm_rows = sum(1 for _, _, t in rows if t == 'rhythm')
    expected_leads = standard_rows * 4 + rhythm_rows
    print(f"\n📊 Expected structure:")
    print(f"   Standard rows: {standard_rows} (4 leads each = {standard_rows * 4} leads)")
    print(f"   Rhythm strips: {rhythm_rows}")
    print(f"   Total leads to extract: {expected_leads}")


print("✅ Row detection functions defined")

In [ ]:
print("\n" + "="*70)
print("🔧 STEP 3: COLUMN SEGMENTATION")
print("="*70)

def crop_lead_region_smart(lead_img, margin_top=50, margin_sides=20):
    """
    Crop lead region to remove labels and margins
    
    Labels are typically in top-left of each cell, so we crop:
    - Top: Remove first 50px (where labels like "I", "aVR" are)
    - Sides: Remove 20px each side (margins)
    
    Args:
        lead_img: Image of a single lead region
        margin_top: Pixels to crop from top (remove labels)
        margin_sides: Pixels to crop from each side
    
    Returns:
        Cropped image with only ECG signal
    """
    h, w = lead_img.shape
    
    # Ensure margins don't exceed image size
    margin_top = min(margin_top, h // 4)
    margin_sides = min(margin_sides, w // 10)
    
    # Crop
    cropped = lead_img[margin_top:h-5, margin_sides:w-margin_sides]
    
    return cropped


def segment_row_into_leads(row_img, n_leads=4):
    """
    Segment a row into individual lead columns WITH smart cropping
    
    Args:
        row_img: Image of a single row
        n_leads: Expected number of leads in this row
    
    Returns:
        lead_regions: List of (x_start, x_end, cropped_img)
    """
    h, w = row_img.shape
    lead_width = w // n_leads
    
    regions = []
    for i in range(n_leads):
        x_start = i * lead_width
        x_end = (i + 1) * lead_width if i < n_leads - 1 else w
        
        # Extract lead region
        lead_img = row_img[:, x_start:x_end]
        
        # Smart crop to remove labels
        lead_cropped = crop_lead_region_smart(lead_img, margin_top=50, margin_sides=20)
        
        regions.append((x_start, x_end, lead_cropped))
    
    return regions


def visualize_lead_segmentation(img, rows, lead_assignments):
    """Visualize final lead segmentation WITH cropped regions"""
    fig, axes = plt.subplots(2, 1, figsize=(20, 14))
    
    # Top: Full image with bounding boxes
    img_vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]
    
    for row_idx, (y_start, y_end, row_type) in enumerate(rows):
        if row_idx >= len(lead_assignments):
            continue
            
        n_leads, lead_names = lead_assignments[row_idx]
        row_img = img[y_start:y_end, :]
        lead_regions = segment_row_into_leads(row_img, n_leads=n_leads)
        
        for col_idx, (x_start, x_end, lead_cropped) in enumerate(lead_regions):
            if col_idx >= len(lead_names):
                continue
                
            color = colors[col_idx % len(colors)]
            
            # Draw FULL region box
            cv2.rectangle(img_vis, (x_start, y_start), (x_end, y_end), color, 2)
            
            # Draw CROPPED region box (inner)
            margin_top = 50
            margin_sides = 20
            cv2.rectangle(img_vis, 
                         (x_start + margin_sides, y_start + margin_top), 
                         (x_end - margin_sides, y_end - 5), 
                         (0, 255, 255), 2)  # Cyan for cropped region
            
            # Add label
            lead_name = lead_names[col_idx]
            cv2.putText(img_vis, lead_name, (x_start + 10, y_start + 25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            cv2.putText(img_vis, '(cropped)', (x_start + 10, y_end - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
    
    axes[0].imshow(img_vis)
    axes[0].set_title('Full Segmentation (Colored boxes = full region, Cyan boxes = cropped signal area)', 
                     fontweight='bold', fontsize=12)
    axes[0].axis('off')
    
    # Bottom: Show some cropped examples
    # Take first 4 leads from row 1 as examples
    if len(rows) > 0 and len(lead_assignments) > 0:
        y_start, y_end, _ = rows[0]
        n_leads, lead_names = lead_assignments[0]
        row_img = img[y_start:y_end, :]
        lead_regions = segment_row_into_leads(row_img, n_leads=min(4, n_leads))
        
        # Concatenate cropped leads horizontally
        cropped_leads = [lead_cropped for _, _, lead_cropped in lead_regions[:4]]
        if cropped_leads:
            composite = np.hstack(cropped_leads)
            axes[1].imshow(composite, cmap='gray')
            axes[1].set_title('Example: Cropped Lead Regions (Row 1, first 4 leads) - Labels removed', 
                             fontweight='bold', fontsize=12)
            axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Cropping Strategy:")
    print("  - Top margin: 50px removed (eliminates labels like 'I', 'aVR', etc.)")
    print("  - Side margins: 20px each side removed")
    print("  - Bottom margin: 5px removed")
    print("  - Cyan boxes show actual signal extraction area")

print("✅ Column segmentation functions defined")

In [ ]:
print("\n" + "="*70)
print("🔄 COMPLETE ADAPTIVE PIPELINE")
print("="*70)

def preprocess_adaptive(image_path, target_size=(1400, 2000), visualize=True):
    """
    Complete adaptive preprocessing pipeline
    
    Returns:
        dict with processed image and metadata
    """
    # Load
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        img = np.array(Image.open(image_path).convert('L'))
    
    # Resize
    if target_size:
        img = cv2.resize(img, (target_size[1], target_size[0]), 
                        interpolation=cv2.INTER_AREA)
    
    print("\n🔧 STEP 1: DESKEWING...")
    # Detect and correct skew
    angle, debug_skew = detect_skew_angle(img)
    img_deskewed = deskew_image(img, angle)
    
    if visualize:
        visualize_deskew(img, img_deskewed, debug_skew)
    
    print("\n🔧 STEP 2: ROW DETECTION...")
    # Detect rows with improved smart method (v2 with text filtering)
    rows, debug_rows = detect_rows_smart_v2(img_deskewed, n_expected_rows=4)
    
    if visualize:
        visualize_row_detection(img_deskewed, rows, debug_rows)
    
    return {
        'original': img,
        'deskewed': img_deskewed,
        'rows': rows,  # Now includes row_type
        'calibration': CALIBRATION,
        'skew_angle': angle
    }

print("✅ Adaptive pipeline defined")

In [ ]:
print("\n" + "="*70)
print("🧪 TESTING ADAPTIVE PIPELINE")
print("="*70)

sample_id = train_meta['id'].iloc[0]
sample_img_path = TRAIN_PATH / str(sample_id) / f"{sample_id}-0001.png"

print(f"Testing on: {sample_id}")
print(f"Image: {sample_img_path.name}")

if sample_img_path.exists():
    print("\n" + "="*70)
    print("🎬 RUNNING ADAPTIVE PIPELINE WITH VISUALIZATION")
    print("="*70)
    
    # Run adaptive preprocessing
    result = preprocess_adaptive(sample_img_path, visualize=True)
    
    print("\n✅ Adaptive preprocessing complete!")
    print(f"\n📊 Results:")
    print(f"   Skew corrected: {result['skew_angle']:.3f}°")
    print(f"   Content rows detected: {len(result['rows'])}")
    
    # Visualize lead assignment (based on detected rows)
    print("\n🔧 STEP 3: LEAD ASSIGNMENT...")
    
    # Determine lead assignment based on number of rows
    n_rows = len(result['rows'])
    
    if n_rows == 3:
        # Standard: 2 rows of 4 leads + 1 rhythm strip
        lead_assignments = [
            (4, ['I', 'aVR', 'V1', 'V4']),
            (4, ['II', 'aVL', 'V2', 'V5']),
            (4, ['III', 'aVF', 'V3', 'V6']),
        ]
    elif n_rows == 4:
        # With rhythm strip separate
        lead_assignments = [
            (4, ['I', 'aVR', 'V1', 'V4']),
            (4, ['II', 'aVL', 'V2', 'V5']),
            (4, ['III', 'aVF', 'V3', 'V6']),
            (1, ['II_rhythm'])
        ]
    else:
        print(f"⚠️  Unexpected number of rows: {n_rows}")
        lead_assignments = []
    
    if lead_assignments:
        print(f"\n📋 Lead Assignment Plan:")
        for row_idx, (n_leads, names) in enumerate(lead_assignments):
            print(f"   Row {row_idx+1}: {n_leads} leads → {names}")
        
        visualize_lead_segmentation(result['deskewed'], result['rows'], lead_assignments)

print("\n" + "="*70)
print("✨ ADAPTIVE DETECTION V3 READY")
print("="*70)
print("\n💡 Key Improvements:")
print("  ✅ Automatic skew detection and correction")
print("  ✅ Adaptive row detection (no fixed layout)")
print("  ✅ Content-aware filtering (skip empty regions)")
print("  ✅ Visual debugging at each step")
print("\n🎯 Next: Implement signal extraction for adaptive layout")
print("   Then we can complete the pipeline!")

In [ ]:
"""
CELDA DE PRUEBA: Probar detección adaptativa en múltiples imágenes
==================================================================
Usa esta celda para probar el pipeline V3 en diferentes muestras
"""

import numpy as np
import pandas as pd
from pathlib import Path

# ============================================================================
# SELECCIÓN DE IMÁGENES PARA PROBAR
# ============================================================================

print("🔍 PRUEBA DE DETECCIÓN EN MÚLTIPLES IMÁGENES")
print("=" * 70)

# Cargar metadata
train_meta = pd.read_csv(DATA_PATH / 'train.csv')

# Mostrar primeras 20 imágenes disponibles
print("\n📋 Imágenes disponibles para probar:")
print(f"{'ID':<12} {'fs (Hz)':<10} {'sig_len':<10}")
print("-" * 35)
for idx, row in train_meta.head(20).iterrows():
    print(f"{row['id']:<12} {row['fs']:<10} {row['sig_len']:<10}")

print("\n" + "=" * 70)
print("💡 INSTRUCCIONES:")
print("   1. Elige un ID de la lista arriba")
print("   2. Copia y pega en test_image_id abajo")
print("   3. Ejecuta la celda")
print("   4. Revisa si la detección funciona bien")
print("=" * 70)

# ============================================================================
# CONFIGURACIÓN DE PRUEBA
# ============================================================================

# 🔧 CAMBIA ESTE ID PARA PROBAR DIFERENTES IMÁGENES
test_image_id = 32650710  # ← Cambia este número

# Configuración
use_variant = '0001'  # Usar imagen limpia (-0001)
visualize_full = True  # Mostrar todas las visualizaciones

# ============================================================================
# EJECUTAR PRUEBA
# ============================================================================

print(f"\n🧪 PROBANDO IMAGEN: {test_image_id}")
print("=" * 70)

# Verificar que existe
if test_image_id not in train_meta['id'].values:
    print(f"❌ ERROR: ID {test_image_id} no existe en el dataset")
    print(f"   IDs disponibles: {train_meta['id'].values[:10]}...")
else:
    # Obtener info
    img_info = train_meta[train_meta['id'] == test_image_id].iloc[0]
    
    print(f"📊 Información de la imagen:")
    print(f"   ID: {img_info['id']}")
    print(f"   Sampling frequency: {img_info['fs']} Hz")
    print(f"   Signal length: {img_info['sig_len']} samples")
    print(f"   Duration: {img_info['sig_len'] / img_info['fs']:.1f} seconds")
    
    # Path de imagen
    img_path = TRAIN_PATH / str(test_image_id) / f"{test_image_id}-{use_variant}.png"
    
    if not img_path.exists():
        print(f"\n❌ Imagen no encontrada: {img_path}")
        print(f"   Verificando qué variantes existen...")
        img_dir = TRAIN_PATH / str(test_image_id)
        if img_dir.exists():
            available = sorted(img_dir.glob(f"{test_image_id}-*.png"))
            print(f"   Variantes disponibles: {[p.name for p in available]}")
    else:
        print(f"\n✅ Imagen encontrada: {img_path.name}")
        
        # ====================================================================
        # EJECUTAR PIPELINE ADAPTATIVO
        # ====================================================================
        
        print("\n" + "=" * 70)
        print("🎬 EJECUTANDO PIPELINE ADAPTATIVO V3")
        print("=" * 70)
        
        try:
            # Ejecutar pipeline completo
            result = preprocess_adaptive(img_path, visualize=visualize_full)
            
            print("\n" + "=" * 70)
            print("✅ PRUEBA COMPLETADA")
            print("=" * 70)
            
            # Resumen de resultados
            print(f"\n📊 RESULTADOS:")
            print(f"   Rotación detectada: {result['skew_angle']:.3f}°")
            print(f"   Filas detectadas: {len(result['rows'])}")
            
            for idx, (y_start, y_end, row_type) in enumerate(result['rows']):
                height = y_end - y_start
                print(f"   Row {idx+1} ({row_type:>8}): y={y_start:>4}-{y_end:>4} (height: {height:>3}px)")
            
            # Análisis de calidad
            print(f"\n🔍 ANÁLISIS DE CALIDAD:")
            
            # Check 1: Número de filas
            n_rows = len(result['rows'])
            if n_rows == 4:
                print(f"   ✅ Número de filas correcto: {n_rows}")
            elif n_rows == 3:
                print(f"   ⚠️  Número de filas: {n_rows} (esperado 4, pero 3 es válido si rhythm integrado)")
            else:
                print(f"   ❌ Número de filas inesperado: {n_rows} (esperado 3-4)")
            
            # Check 2: Altura de filas
            heights = [y_end - y_start for y_start, y_end, _ in result['rows']]
            standard_heights = [h for (_, _, t), h in zip(result['rows'], heights) if t == 'standard']
            rhythm_heights = [h for (_, _, t), h in zip(result['rows'], heights) if t == 'rhythm']
            
            if standard_heights:
                avg_standard = np.mean(standard_heights)
                std_standard = np.std(standard_heights)
                print(f"   Standard rows: avg height = {avg_standard:.1f}px ± {std_standard:.1f}px")
                
                if std_standard < 50:
                    print(f"   ✅ Standard rows tienen altura consistente")
                else:
                    print(f"   ⚠️  Standard rows varían mucho en altura")
            
            if rhythm_heights:
                print(f"   Rhythm strip: height = {rhythm_heights[0]:.1f}px")
                if rhythm_heights[0] > avg_standard * 1.5:
                    print(f"   ✅ Rhythm strip es significativamente más grande")
                else:
                    print(f"   ⚠️  Rhythm strip no es mucho más grande que standard rows")
            
            # Check 3: Rotación
            if abs(result['skew_angle']) < 0.5:
                print(f"   ✅ Imagen está bien alineada (rotación mínima)")
            elif abs(result['skew_angle']) < 2.0:
                print(f"   ✅ Rotación leve corregida exitosamente")
            else:
                print(f"   ⚠️  Rotación significativa: {result['skew_angle']:.2f}°")
            
            print("\n" + "=" * 70)
            print("💡 INTERPRETACIÓN:")
            print("   - Si ves 4 filas bien divididas → Detección exitosa")
            print("   - Si cajas cyan capturan solo señal (sin labels) → Crop exitoso")
            print("   - Si hay problemas, prueba con otro ID")
            print("=" * 70)
            
        except Exception as e:
            print(f"\n❌ ERROR durante la ejecución:")
            print(f"   {type(e).__name__}: {str(e)}")
            import traceback
            print("\n📋 Traceback completo:")
            traceback.print_exc()

# ============================================================================
# PRUEBA RÁPIDA EN BATCH (OPCIONAL)
# ============================================================================

print("\n" + "=" * 70)
print("🔬 PRUEBA RÁPIDA EN BATCH (OPCIONAL)")
print("=" * 70)
print("Descomenta el código abajo para probar automáticamente en 5 imágenes")

"""
# Lista de IDs para probar
test_ids = train_meta['id'].head(5).values

batch_results = []

for test_id in test_ids:
    img_path = TRAIN_PATH / str(test_id) / f"{test_id}-0001.png"
    
    if not img_path.exists():
        continue
    
    try:
        # Ejecutar sin visualización
        result = preprocess_adaptive(img_path, visualize=False)
        
        batch_results.append({
            'id': test_id,
            'skew_angle': result['skew_angle'],
            'n_rows': len(result['rows']),
            'success': True
        })
        
    except Exception as e:
        batch_results.append({
            'id': test_id,
            'skew_angle': None,
            'n_rows': None,
            'success': False,
            'error': str(e)
        })

# Mostrar resultados
batch_df = pd.DataFrame(batch_results)
print("\n📊 Resultados de prueba en batch:")
print(batch_df)

success_rate = (batch_df['success'].sum() / len(batch_df)) * 100
print(f"\n✅ Tasa de éxito: {success_rate:.1f}%")
"""

print("\n✨ Listo para probar más imágenes!")
print("   Cambia 'test_image_id' arriba y vuelve a ejecutar")